## Retrieve the gauge data and plot using WoFS
Now we have about 300 reservoirs that are linked to their gauges. I want to be able to retrieve Water Observations from Space satellite images of each reservoir at every 1m depth interval. This way, we can see the surface area of the reservoir according to the depth of the gauge. Making a relationship between 

In [1]:
%matplotlib inline

import os
import sys
import pickle
import datacube
import numpy as np
import pandas as pd
from pandas import DataFrame
import geopandas as gpd
import geopandas.tools
from shapely.geometry import Point
import xarray as xr
import ipyleaflet as ipyl
from ipyleaflet import Map, Marker
import ipywidgets as ipyw
import json
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display
from datacube.utils import masking
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff

I did a spatial join in ArcGIS of the named_reservoirs shapefile and the gauge locations that I got from BoM. the Distance column is how far each gauge is from the reservoir. A value of 0 means the gauge is inside the reservoir. There are some reservoirs like Jindabyne that have more than one polygon and the distance to the gauge is like 0.055398, but it is the right gauge. However I'm going to only take distance values under 0.011 because from looking at the attribute table in ArcGIS, that's the distance where each gauge is true to each reservoir. If you take higher than that you start getting reservoirs that are matched to the wrong gauge. 

In [2]:
join = gpd.read_file('points_polys_join/points_polys_join.shp')
join_matched = join[join['Distance'] <= 0.011]
join_matched = join_matched.drop(columns=['FID_1', 'FID_2'])
join_matched

,NAME,SHAPE_Area,Area_calc,lat,lon,staion_nam,lat_1,lon_1,Distance,geometry
0,TEN MILE BROOK DAM,0.000029,2.941558e+05,-33.963056,115.125976,"10 Mile Brook Dam Water Level Daily Value, Sou...",-33.963617,115.124899,0.000000,"POLYGON ((115.12337 -33.95985, 115.12344 -33.9..."
1,LAKE ALBERT,0.018549,1.864490e+08,-35.618794,139.283804,2km N Warringee Pt,-35.676602,139.250269,0.000000,"POLYGON ((139.25587 -35.63729, 139.25642 -35.6..."
2,LAKE ALEXANDRINA,0.061854,6.230347e+08,-35.449258,139.149024,3km W Pt McLeay,-35.514813,139.064752,0.000000,"POLYGON ((139.36075 -35.36765, 139.36370 -35.3..."
3,ARTHURS LAKE,0.006971,6.417353e+07,-41.980583,146.923891,ARTHURS LAKE - AT PUMP STATION,-41.988306,146.862472,0.000000,"POLYGON ((146.90975 -41.93936, 146.90994 -41.9..."
4,LAKE AUGUSTA,0.001265,1.166882e+07,-41.853706,146.530548,AUGUSTA LAKE - AT INTAKE,-41.844444,146.572222,0.000000,"POLYGON ((146.55101 -41.83153, 146.55113 -41.8..."
...,...,...,...,...,...,...,...,...,...,...
663,YAN YEAN RESERVOIR,0.000502,4.923370e+06,-37.555279,145.137892,YAN YEAN RES HG,-37.552300,145.129000,0.000000,"POLYGON ((145.14773 -37.55130, 145.14775 -37.5..."
664,UPPER YARRA RESERVOIR,0.000747,7.311508e+06,-37.687427,145.926259,YARRA RIV-UY RES HG,-37.673072,145.900960,0.000000,"POLYGON ((145.89486 -37.67682, 145.89512 -37.6..."
665,LAKE BORUMBA,0.000388,4.283537e+06,-26.520656,152.559265,Yabba Ck at Borumba Dam HW,-26.509675,152.581865,0.000264,"POLYGON ((152.53380 -26.51082, 152.53482 -26.5..."
668,LAKE MULWALA,0.003668,3.669286e+07,-36.000460,146.071580,Yarrawonga Weir,-36.010308,146.091535,0.000000,"POLYGON ((146.04320 -35.96721, 146.04335 -35.9..."


In [3]:
#Converting the data to json
data = json.loads(join_matched.to_json())

map = ipyl.Map(center=[-28, 148], zoom=7)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

#This part draws the polygons for the reservoirs
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

def click_handler(event=None, feature=None, id=None, properties=None):
    label.value = str(properties['NAME'])
    
    
layer.on_hover(click_handler)
map.add_layer(layer)
map

Map(center=[-28, 148], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…